# PDF 크롤링 테스트

In [1]:
import pandas as pd

In [3]:
# !pip install PyMuPDF
# import fitz

In [5]:
def extract_text_from_pdf(pdf_path: str) -> str :

    doc = fitz.open(pdf_path)    # PDF 열기
    full_text = []

    for page_num, page in enumerate(doc, start=1):
        text = page.get_text()   # 페이지의 텍스트 추출
        full_text.append(f"{text}\n")

    doc.close()
    return "".join(full_text)


if __name__ == "__main__":
    pdf_file = "/content/drive/MyDrive/Hanhwa/crawling_project/crawling_project/data/국방과학기술플러스 262호.pdf"  # 텍스트를 추출할 PDF 파일 경로
    extracted_text = extract_text_from_pdf(pdf_file)
    # print(extracted_text)

In [7]:
import re
def split_sentences(text):
    # 마침표·물음표·느낌표 뒤에 공백이 올 때만 분리
    sentences = re.split(r'(?<=[\.!?])\s+', text)
    # 짧은 조각(10자 미만)은 제거
    sentences = [s for s in sentences if len(s) > 10]
    return sentences

In [8]:
example = split_sentences(extracted_text)
for i in range(len(example)):
    example[i] = example[i].replace('\n','')

In [20]:
print(len(example))
example[53]

952


'그림 3과 같이 모델 전체를 CNN이나 Transformer로 구성할 수 있으며, Transformer를 백본 네트워크로 사용하고 CNN 구조의 탐지기를 활용하거나 CNN 백본 네트 워크를 사용하고 Transformer decoder를 활용하는 등 두 (a) 2-Stage 탐지 모델(b) 1-Stage 탐지 모델감시정찰용 표적 탐지/인식을 위한 인공지능 기술 | 7모델을 융합할 수도 있다.'

In [49]:
def extract_text_from_pdf(pdf_path: str) -> str:
    """
    PDF의 모든 페이지에서 텍스트를 뽑아 하나의 문자열로 반환
    """
    doc = fitz.open(pdf_path)
    full_text = []
    for page in doc:
        full_text.append(page.get_text() + "\n")
    doc.close()
    return "".join(full_text)

def normalize_linebreaks(text: str) -> str:
    """
    • 단일 \n은 문장 연결용으로 간주하고 공백으로 대체
    • 연속된 \n\n 이상은 그대로 두어 단락 구분 유지
    """
    return re.sub(r'(?<!\n)\n(?!\n)', '', text)

def split_sentences(text: str, min_len: int = 10) -> list[str]:
    """
    • normalize_linebreaks로 먼저 연결
    • 영문 구두점(.!?)+공백 or 개행,
      한글 종결어미(다/요/습니다 등)+개행에 따라 분리
    • 너무 짧은 조각은 제거
    """
    # 1) 중간 줄바꿈 정리
    text = normalize_linebreaks(text)

    # 2) 분리 패턴
    pattern = (
        r'(?<=[\.!?])(?:\s+|\n)'   # .?! 뒤의 스페이스나 개행
        r'|(?<=[다요습니다요다])\n' # 한글 종결어미 뒤의 개행
    )
    parts = re.split(pattern, text)

    # 3) 10자 미만 제거 + 양쪽 공백 strip
    return [p.strip() for p in parts if len(p.strip()) >= min_len]

if __name__ == "__main__":
    pdf_file = "/content/drive/MyDrive/Hanhwa/crawling_project/crawling_project/data/국방과학기술플러스 262호.pdf"
    raw = extract_text_from_pdf(pdf_file)
    sentences = split_sentences(raw)

# 용어와 용례 매칭 테스트

- 매핑시킬 단어 사전

In [72]:
weapon = pd.read_csv('/content/drive/MyDrive/Hanhwa/crawling_project/crawling_project/data/무기의세계/무기의세계_불필요한것 제외.csv')
weapon.head()

,용어,설명
0,F-2,F-2는 일본 항공자위대가 운용중인 전투기로 F-16 전투기를 바탕으로 미국과 일본...
1,알레이버크급 구축함,알레이버크급 구축함은 미 해군의 주력 수상전투함으로 이지스함의 대명사로 알려져 있다...
2,Ju 52 수송기,Ju 52는 제2차 대전 관련 에피소드에서 약방의 감초처럼 등장하는 숨은 주인공이었...
3,X-37B,X-37B는 미 공군이 운용중인 무인우주선으로 궤도시험기라는 정식명칭을 가지고 있다...
4,미니트맨,미니트맨은 미 공군이 운용중인 대륙간탄도미사일로 미국의 핵 억제력에 중추적인 위치를...


In [74]:
weapon_term = list(weapon['용어'])
len(weapon_term)

366

In [87]:
map_dict = {}
not_map_dict = {}

# 2) term 별로 예시 수집
for term in weapon_term:
    matched = [ex for ex in sentences if term in ex]
    if matched:
        map_dict[term] = matched
    else:
        not_map_dict[term] = []  # 나중에 필요하시면 비어 있는 대신 count, None 등으로 바꿔도 OK

# 3) 결과 확인
print(f"매칭된 용어 수: {len(map_dict)}")
print(f"매칭되지 않은 용어 수: {len(not_map_dict)}")

매칭된 용어 수: 3
매칭되지 않은 용어 수: 363


In [91]:
map_dict

{'SM-3': ['국방과학연구소 국방첨단과학기술연구원 소재에너지센터 선임연구원 이정민 온도, 압력, 유량↑ \uf0e8삭마↑ X43-A 온도, 압력, 유량↑ \uf0e8강도↓, 기밀↓ 능동냉각유로 리딩엣지 스크램젯 온도, 압력, 유량↑ \uf0e8삭마↑ SM-3 노즐목 위치•자세제어 Pue, A.',
  '탄소섬유 기반 초고온 내열/내산화 복합소재 기술   \t 1\t 개요 초고온 내열 복합소재는 위에 대표 그림에서 나타나  있는 것과 같이 사드(THADD)와 유사한 유도탄 방어 무기체계(SM-3)에 필수 핵심부품 소재이다.'],
 '사드': ['탄소섬유 기반 초고온 내열/내산화 복합소재 기술   \t 1\t 개요 초고온 내열 복합소재는 위에 대표 그림에서 나타나  있는 것과 같이 사드(THADD)와 유사한 유도탄 방어 무기체계(SM-3)에 필수 핵심부품 소재이다.'],
 '어뢰': ['러시 아의 초공동 어뢰인 쉬크발(Shkval)의 경우 초공동 현 상 발생장치와 해수흡입 추진기관을 조합하여 약 200 노트의 속도를 내는 것으로 알려져 있다.',
  '전기모터 혹 은 내연기관 방식의 어뢰가 50노트 수준임을 고려하면  표적이 회피하거나 반응할 시간을 제한하는 개념의 무 수중 유도무기 패러다임의 변화 해수흡입 추진기관!',
  '해수흡입 추진기관 기술 동향 [그림 1] 초공동-해수흡입 추진기관 적용 어뢰 (A) 운용개념 (B) 이란 후트(Hoot) 발사 장면\n\n32 | 국방과학기술플러스 + 기체계이다.',
  '하지만 이러한 초공동 어뢰의 경우 소음이 크고, 상 대적으로 사거리가 짧으며 유도기능이 제한적이라는  단점이 존재한다.',
  '이러한 문제를 보완하기 위하여 그림  1과 같이 중요한 시설 보호를 위하여 고정식 발사 플 랫폼을 활용하거나 수상함에서 발사하는 개념이 제안 되고 있으며, 이란의 초공동 어뢰인 후트(Hoot)의 경 우 수상함에서 시험 발사하는 장면을 확인할 수 있다.',
  '초공동 어뢰는  발사관에서 발사된 후, 사출모타를 통해 초기 가

# 용어와 매칭 함수

In [94]:
# !pip install PyMuPDF

In [95]:
import os
import re
import fitz          # PyMuPDF
import pandas as pd

def extract_text_from_pdf(pdf_path: str) -> str:
    doc = fitz.open(pdf_path)
    full_text = "".join(page.get_text() + "\n" for page in doc)
    doc.close()
    return full_text

def normalize_linebreaks(text: str) -> str:
    text = re.sub(r'(?<!\n)\n(?!\n)', '', text)
    text = re.sub(r'(?<=[\.!?])(?=[^\s])', ' ', text) # ? . ! 뒤에 공백이 없는 경우 의도적으로 붙여주기
    return text

def split_sentences(text: str, min_len: int = 10) -> list[str]:
    text = normalize_linebreaks(text)
    pattern = r'(?<=[\.!?])(?:\s+|\n)|(?<=[다요었습니다요다])\n'
    parts = re.split(pattern, text)
    return [p.strip() for p in parts if len(p.strip()) >= min_len]

def map_terms_to_sentences(terms: list[str], sentences: list[str]) -> dict[str, list[str]]:
    mapped = {}
    for term in terms:
        hits = [s for s in sentences if term in s]
        if hits:
            mapped[term] = list(dict.fromkeys(hits))
    return mapped

def combine_pdfs_to_one_csv(pdf_dir: str,
                            terms: list[str],
                            output_csv: str,
                            min_len: int = 10):
    # 1) pdf 폴더에서 .pdf 파일들 자동 수집
    pdf_files = [
        os.path.join(pdf_dir, fn)
        for fn in os.listdir(pdf_dir)
        if fn.lower().endswith(".pdf")
    ]

    combined_rows = []
    # 2) PDF별 처리
    for pdf_path in pdf_files:
        base = os.path.splitext(os.path.basename(pdf_path))[0]
        raw = extract_text_from_pdf(pdf_path)
        sentences = split_sentences(raw, min_len=min_len)
        mapped = map_terms_to_sentences(terms, sentences)

        for term, examples in mapped.items():
            for ex in examples:
                combined_rows.append({
                    'pdf': base,
                    'term': term,
                    'example': ex
                })
        print(f" • {base}.pdf → {len(mapped)}개 용어 매핑")


    df_all = pd.DataFrame(combined_rows)

    # 3-1) “term + example” 쌍 단위로 중복 제거
    df_all = df_all.drop_duplicates(subset=['term', 'example'])

    df_all = df_all.sort_values(by='term')

    # 4) 용례 개수 확인
    total_examples = len(df_all)
    print(f"\n▶ 총 용례 개수: {total_examples}건\n")

    print("▶ 용어별 용례 개수:")
    print(df_all['term'].value_counts(), "\n")

    print("▶ PDF별 용례 개수:")
    print(df_all['pdf'].value_counts(), "\n")

    # 5) CSV로 저장
    df_all.to_csv(output_csv, index=False, encoding="utf-8-sig")
    print(f"✔ 최종 결과가 '{output_csv}' 에 저장되었습니다.")

In [96]:
if __name__ == "__main__":
    weapon_df = pd.read_csv(
          '/content/drive/MyDrive/Hanhwa/crawling_project/crawling_project/data/무기의세계/무기의세계_불필요한것 제외.csv',
        encoding="utf-8-sig"
    )
    weapon_terms = weapon_df['용어'].tolist()

    pdf_folder =  "/content/drive/MyDrive/Hanhwa/crawling_project/crawling_project/data/"
    output_file ="/content/drive/MyDrive/Hanhwa/crawling_project/crawling_project/data/test.csv"

    combine_pdfs_to_one_csv(pdf_folder, weapon_terms, output_file)

 • 국방과학기술플러스 262호.pdf → 3개 용어 매핑
 • 261 국방과학기술플러스_최종안.pdf → 5개 용어 매핑
 • 국방과학기술플러스 vol.260.pdf → 3개 용어 매핑
 • 259호_국방과학기술플러스.pdf → 9개 용어 매핑
 • 국방과학기술플러스 Vol.257.pdf → 2개 용어 매핑
 • 국방과학기술플러스 Vol.256.pdf → 14개 용어 매핑
 • 국방과학기술플러스 Vol.258.pdf → 8개 용어 매핑
 • 국방과학기술플러스 Vol.255.pdf → 9개 용어 매핑
 • 국방기술플러스 251호.pdf → 3개 용어 매핑
 • 2021_국방과학기술플러스_253호.pdf → 9개 용어 매핑
 • 2021국방과학기술플러스(254호).pdf → 2개 용어 매핑
 • 2021국방과학기술플러스252호.pdf → 2개 용어 매핑

▶ 총 용례 개수: 249건

▶ 용어별 용례 개수:
term
어뢰            69
지뢰            25
정찰기           24
정찰위성          20
F-2           13
전자전기          12
미사일 방어체계      11
기뢰             8
견인포            7
장갑차            7
함포             7
기총             6
SM-3           5
프레데터           4
대륙간탄도미사일       3
장사정포           3
사드             3
상륙돌격장갑차        2
T-50           2
C-130 허큘리스     1
5세대 전투기        1
J-20 

# 전처리1 : 정확한 용례만 남기기

In [97]:
import pandas as pd
weapon_ex = pd.read_csv('/content/drive/MyDrive/Hanhwa/crawling_project/crawling_project/data/test.csv')
weapon_ex.head()

,pdf,term,example
0,국방과학기술플러스 Vol.255,5세대 전투기,■ PL-10 (중국)중국에서 2004년부터 5세대 전투기용으로 개발한 5세대 단...
1,국방과학기술플러스 Vol.256,C-130 허큘리스,"전술수송기의 결정판 C-130 허큘리스, 2017."
2,국방과학기술플러스 Vol.255,F-2,"장착기종 으로 F-2, F-15J, F-4EJ에서 운용 중이며, 정밀도 높은 RLG..."
3,국방과학기술플러스 Vol.255,F-2,"이후 KF-21 체계개발이 확정되고, 국내 항공무장통합 환경이 점차 갖춰짐에 따라 ..."
4,국방과학기술플러스 Vol.255,F-2,"미국산 최신 전투기에 장착이 가능하며, 한국형전투기 (KF-21) 장착 무장으로 고..."


In [98]:
import re

def filter_examples(terms, examples):
    """
    terms: 용어 리스트 (iterable of str)
    examples: 모든 용례가 담긴 리스트
    반환: { term: [매칭된 용례들], … }
    """
    matched_dict = {}
    for term in terms:
        # term 바로 뒤에 숫자가 오면 매칭에서 제외
        pattern = re.compile(r'(?<!\d)' + re.escape(term) + r'(?!\d)') # 앞 뒤에 숫자가 못오게 막기
        matched_dict[term] = [ex for ex in examples if pattern.search(ex)]
    return matched_dict

In [99]:
# 1) all_examples 구성
all_examples = []
for row in weapon_ex['example']:
    try:
        all_examples.append(row)
    except:
        continue

# 2) terms 불러오기
terms = weapon_ex['term'].unique().tolist()

# 3) 필터링 수행
filtered_dict = filter_examples(terms, all_examples)

# 4) 중복값 제거
for term, exs in filtered_dict.items():
    # dict.fromkeys 이용해 순서는 유지하되 중복 값은 하나만 남김
    filtered_dict[term] = list(dict.fromkeys(exs))

In [100]:
# 1) 용례가 있는 용어만 필터링
terms_with_examples = {
    term: exs
    for term, exs in filtered_dict.items()
    if exs  # 리스트가 비어있지 않은 경우만
}
len(terms_with_examples.keys()) # 37개

37

In [101]:
# 2) 확인하기
print("용례가 있는 용어와 개수:")
for term, exs in terms_with_examples.items():
    print(f"{term}: {len(exs)}건 -> {exs}")

용례가 있는 용어와 개수:
5세대 전투기: 1건 -> ['■ PL-10 (중국)중국에서 2004년부터 5세대 전투기용으로 개발한  5세대 단거리공대공유도탄이다.']
C-130 허큘리스: 1건 -> ['전술수송기의 결정판 C-130 허큘리스, 2017.']
F-2: 1건 -> ['장착기종 으로 F-2, F-15J, F-4EJ에서 운용 중이며, 정밀도 높은 RLG(Ring Laser Gyro) IMU를 탑재하여 LOAL 모드 운용이 가능하다.']
F-22 랩터: 1건 -> ['최근에는 국방예산에 있어 전세계 지출규모의 약 40%를 차지하는 미국에서 조차도 서울에서 개최한 ‘ADEX 2015’에서 전시 및 시험비행에 참여한 적이 있는 세계 최강의 스텔스 전투기로 평가받는 ‘F-22 랩터(Raptor)’의 과도한 획득 및 운영유지 비용으로 인하여 조기퇴역하고 대체할 차세대전투기를 고려하고 있다고 미공군 참모총장이 최근 개최된 맥앨리스 컨퍼런스에서 언급하였다.']
J-20: 1건 -> ['중국의 최신전투기인 J-10, J-11, J-20, FC-31 전투기에 장착 예정이며  수출용 버전인 PL-10E도 개발하였다.']
P-3C: 1건 -> ['하지만, 이스라엘의 ELTA, IAI, Rafael 사의 장비에 의존하여 RF-16펠콘 유인정찰기, 송골매 무인정찰기, P-3C 성능개량기 등을 운용하였다.']
SM-3: 5건 -> ['[표 3] AEGIS 베이스라인별 보유 능력AEGIS B/L탑재 플랫폼주요 능력비  고B/L 5CG 59-65, 67,68,70-73 / DDG 51-78•AAW•MIL-SPEC 적용통합 AAW, 전장상황인식 능력 향상B/L 6CG 66, 69 / DDG79-90•AAW•MIL-SPEC/COTS 적용•Link-16, CEC네트워크 향상B/L 7DDG 91-112• AAW•COTS 적용•Link-16, CEC구역 대공방어 능력 향상B/L 8CG 52-58•B/L 7과 동일B/L 99ACG 59-64•AAW, CEC, NIFC-CA, SM-2, SM-

In [102]:
total_matched = sum(len(v) for v in filtered_dict.values())
print(f"전체 매칭된 용례 개수: {total_matched}건")

전체 매칭된 용례 개수: 236건


# 전처리 2 : 무공백 길이가 20이상 제거

In [103]:
def filter_examples(terms, examples, max_run=20):
    """
    terms: 용어 리스트
    examples: 모든 용례 리스트
    max_len: 이 길이 초과 문장은 제외
    max_run: 이 길이 초과 무공백(스페이스 없는) 구간이 있으면 제외
    반환: { term: [매칭된 용례들], … }
    """
    matched_dict = {}
    for term in terms:
        pattern = re.compile(r'(?<!\d)' + re.escape(term) + r'(?!\d)')
        filtered = []
        for ex in examples:

            #  용어 매칭 & 길이 제한
            if not (pattern.search(ex)) :
                continue

            #  “긴 무공백 구간” 검사:
            #    연속된 한글·영문·숫자 구간을 찾아,
            #    그 길이가 max_run 초과하면 제외
            runs = re.findall(r'[가-힣A-Za-z0-9]+', ex) # 공백을 쪼개서 부분 문자열로 만듦
            if any(len(run) > max_run for run in runs):
                continue

            filtered.append(ex) # 무공백이 20 이하인 용례만 append

        # 4) 순서 유지하며 중복 제거
        matched_dict[term] = list(dict.fromkeys(filtered))

    return matched_dict

In [104]:
all_examples = []
for row in weapon_ex['example']:
    try:
        all_examples.append(row)
    except:
        continue

# 2) terms 불러오기
terms = weapon_ex['term'].unique().tolist()

# 3) 필터링 수행
filtered_dict = filter_examples(terms, all_examples, max_run=20)

# 4) 중복값 제거
for term, exs in filtered_dict.items():
    # dict.fromkeys 이용해 순서는 유지하되 중복 값은 하나만 남김
    filtered_dict[term] = list(dict.fromkeys(exs))

In [105]:
have_example_term = [term for term, exs in filtered_dict.items() if exs]
print(have_example_term) # 용례가 있는 용어
num_terms_with_examples = sum(1 for exs in filtered_dict.values() if exs)
num_examples_with_examples = sum(len(exs) for exs in filtered_dict.values() if exs)
print(f"용례가 있는 용어의 개수: {num_terms_with_examples}개") # 37개로 유지
print(f"총 용례의 개수: {num_examples_with_examples}개") # 232개로 감소(236개->232개)

['5세대 전투기', 'C-130 허큘리스', 'F-2', 'F-22 랩터', 'J-20', 'P-3C', 'SM-3', 'T-50', 'X-37B', '견인포', '공격기', '공대공 미사일', '기뢰', '기총', '대륙간탄도미사일', '대포병 레이더', '독도함', '랴오닝호', '무인전투기', '미사일 방어체계', '비살상무기', '사드', '상륙돌격장갑차', '알레이버크급 구축함', '어뢰', '장갑차', '장사정포', '전략 폭격기', '전익기', '전자전기', '정찰기', '정찰위성', '지뢰', '천궁', '프레데터', '함포', '화차']
용례가 있는 용어의 개수: 37개
총 용례의 개수: 232개


In [106]:
# 결과 확인
for term, exs in filtered_dict.items():
    if exs:
        print(f"{term} ({len(exs)}건):")
        for ex in exs:
            print("  -", ex)
        print()

5세대 전투기 (1건):
  - ■ PL-10 (중국)중국에서 2004년부터 5세대 전투기용으로 개발한  5세대 단거리공대공유도탄이다.

C-130 허큘리스 (1건):
  - 전술수송기의 결정판 C-130 허큘리스, 2017.

F-2 (1건):
  - 장착기종 으로 F-2, F-15J, F-4EJ에서 운용 중이며, 정밀도 높은 RLG(Ring Laser Gyro) IMU를 탑재하여 LOAL 모드 운용이 가능하다.

F-22 랩터 (1건):
  - 최근에는 국방예산에 있어 전세계 지출규모의 약 40%를 차지하는 미국에서 조차도 서울에서 개최한 ‘ADEX 2015’에서 전시 및 시험비행에 참여한 적이 있는 세계 최강의 스텔스 전투기로 평가받는 ‘F-22 랩터(Raptor)’의 과도한 획득 및 운영유지 비용으로 인하여 조기퇴역하고 대체할 차세대전투기를 고려하고 있다고 미공군 참모총장이 최근 개최된 맥앨리스 컨퍼런스에서 언급하였다.

J-20 (1건):
  - 중국의 최신전투기인 J-10, J-11, J-20, FC-31 전투기에 장착 예정이며  수출용 버전인 PL-10E도 개발하였다.

P-3C (1건):
  - 하지만, 이스라엘의 ELTA, IAI, Rafael 사의 장비에 의존하여 RF-16펠콘 유인정찰기, 송골매 무인정찰기, P-3C 성능개량기 등을 운용하였다.

SM-3 (5건):
  - [표 3] AEGIS 베이스라인별 보유 능력AEGIS B/L탑재 플랫폼주요 능력비  고B/L 5CG 59-65, 67,68,70-73 / DDG 51-78•AAW•MIL-SPEC 적용통합 AAW, 전장상황인식 능력 향상B/L 6CG 66, 69 / DDG79-90•AAW•MIL-SPEC/COTS 적용•Link-16, CEC네트워크 향상B/L 7DDG 91-112• AAW•COTS 적용•Link-16, CEC구역 대공방어 능력 향상B/L 8CG 52-58•B/L 7과 동일B/L 99ACG 59-64•AAW, CEC, NIFC-CA, SM-2, SM-6복합임무 구역 대공방어능

# 전처리3 : \n\n이 있는 문장 제거

ex) 관련없는 문장까지 가져와지는 경우 다수
 - 실험은 1) 팔 근력보조 모듈 없이 통상적인 지뢰탐지동작을 하는 경우, 2) 팔 근력보조장치 착용 후 근력보조를 받으며 지뢰탐지동작을 하는 경우, 2가지 경우에 대해서 진행하였다.[그림 17] 지뢰탐지 근력보조 모듈 효과도 분석을 위한 실험 환경[그림 18] 지뢰탐지 근력보조 모듈 사용 여부에 따른 근전도 신호 결과

임무유형별 군사용 착용로봇 플랫폼 개발동향 ▒ 49이에 대한 결과가 그림 18 및 표 6에 제시된다.

- 중력보상부 기구부에서 스프링에 의해 전달되는 보상토크를 기구학적으로 해석하였고 이 보상토크가 중력보상 요구토크와 유사하게 될 수 있도록 기구 파라미터에 대한 최적설계를 수행하였다.[그림 16] 지뢰탐지 근력보조 모듈 중력보상 부 파라미터최적설계

48 ▒ 국방과학기술플러스 +3) 성능검증제작된 지뢰탐지병사용 착용형 팔 근력보조장치 모듈의 효과도를 검증하기 위해 1명의 피험자를 모집하여 실험을 진행하였다.

In [107]:
def filter_examples(terms, examples,
                    max_run: int = 20,) -> dict[str, list[str]]:
    matched_dict = {}
    for term in terms:
        pattern = re.compile(r'(?<!\d)' + re.escape(term) + r'(?!\d)')
        filtered = []
        for ex in examples:

            #  용어 매칭 & 길이 제한
            if not (pattern.search(ex)) :
                continue

            if "\n\n" in ex:
                continue

            # 3) “긴 무공백 구간” 검사 (이전과 동일)
            runs = re.findall(r'[가-힣A-Za-z0-9]+', ex)
            if any(len(run) > max_run for run in runs):
                continue

            filtered.append(ex)


        # 4) 순서 유지하며 중복 제거
        matched_dict[term] = list(dict.fromkeys(filtered))

    return matched_dict

In [109]:
all_examples = []
for row in weapon_ex['example']:
    try:
        all_examples.append(row)
    except:
        continue

# 2) terms 불러오기
terms = weapon_ex['term'].unique().tolist()

# 3) 필터링 수행
filtered_dict = filter_examples(terms, all_examples, max_run=20)

# 4) 중복값 제거
for term, exs in filtered_dict.items():
    # dict.fromkeys 이용해 순서는 유지하되 중복 값은 하나만 남김
    filtered_dict[term] = list(dict.fromkeys(exs))

In [110]:
have_example_term = [term for term, exs in filtered_dict.items() if exs]
print(have_example_term) # 용례가 있는 용어
num_terms_with_examples = sum(1 for exs in filtered_dict.values() if exs)
num_examples_with_examples = sum(len(exs) for exs in filtered_dict.values() if exs)
print(f"용례가 있는 용어의 개수: {num_terms_with_examples}개") # 35개로 감소(37개 -> 35개)
print(f"총 용례의 개수: {num_examples_with_examples}개") # 205개로 감소(236개->232개->205개)

['5세대 전투기', 'C-130 허큘리스', 'F-2', 'F-22 랩터', 'J-20', 'P-3C', 'SM-3', 'T-50', '견인포', '공격기', '공대공 미사일', '기뢰', '기총', '대륙간탄도미사일', '대포병 레이더', '독도함', '랴오닝호', '무인전투기', '미사일 방어체계', '비살상무기', '사드', '상륙돌격장갑차', '알레이버크급 구축함', '어뢰', '장갑차', '장사정포', '전략 폭격기', '전익기', '전자전기', '정찰기', '정찰위성', '지뢰', '천궁', '프레데터', '함포']
용례가 있는 용어의 개수: 35개
총 용례의 개수: 205개


In [111]:
# 결과 확인
for term, exs in filtered_dict.items():
    if exs:
        print(f"{term} ({len(exs)}건):")
        for ex in exs:
            print("  -", ex)
        print()

5세대 전투기 (1건):
  - ■ PL-10 (중국)중국에서 2004년부터 5세대 전투기용으로 개발한  5세대 단거리공대공유도탄이다.

C-130 허큘리스 (1건):
  - 전술수송기의 결정판 C-130 허큘리스, 2017.

F-2 (1건):
  - 장착기종 으로 F-2, F-15J, F-4EJ에서 운용 중이며, 정밀도 높은 RLG(Ring Laser Gyro) IMU를 탑재하여 LOAL 모드 운용이 가능하다.

F-22 랩터 (1건):
  - 최근에는 국방예산에 있어 전세계 지출규모의 약 40%를 차지하는 미국에서 조차도 서울에서 개최한 ‘ADEX 2015’에서 전시 및 시험비행에 참여한 적이 있는 세계 최강의 스텔스 전투기로 평가받는 ‘F-22 랩터(Raptor)’의 과도한 획득 및 운영유지 비용으로 인하여 조기퇴역하고 대체할 차세대전투기를 고려하고 있다고 미공군 참모총장이 최근 개최된 맥앨리스 컨퍼런스에서 언급하였다.

J-20 (1건):
  - 중국의 최신전투기인 J-10, J-11, J-20, FC-31 전투기에 장착 예정이며  수출용 버전인 PL-10E도 개발하였다.

P-3C (1건):
  - 하지만, 이스라엘의 ELTA, IAI, Rafael 사의 장비에 의존하여 RF-16펠콘 유인정찰기, 송골매 무인정찰기, P-3C 성능개량기 등을 운용하였다.

SM-3 (5건):
  - [표 3] AEGIS 베이스라인별 보유 능력AEGIS B/L탑재 플랫폼주요 능력비  고B/L 5CG 59-65, 67,68,70-73 / DDG 51-78•AAW•MIL-SPEC 적용통합 AAW, 전장상황인식 능력 향상B/L 6CG 66, 69 / DDG79-90•AAW•MIL-SPEC/COTS 적용•Link-16, CEC네트워크 향상B/L 7DDG 91-112• AAW•COTS 적용•Link-16, CEC구역 대공방어 능력 향상B/L 8CG 52-58•B/L 7과 동일B/L 99ACG 59-64•AAW, CEC, NIFC-CA, SM-2, SM-6복합임무 구역 대공방어능

# 전처리4 : 연속된 공백이 너무 긴 경우 제거

In [ ]:
weapon_ex = pd.read_csv('/content/drive/MyDrive/Hanhwa/crawling_project/crawling_project/data/test.csv')
weapon_ex.head()

In [116]:
weapon_ex = pd.read_csv('/content/drive/MyDrive/Hanhwa/crawling_project/crawling_project/data/test.csv')

def filter_examples(terms, examples,
                    max_run: int = 20, max_space_run: int = 5, max_len = 100) -> dict[str, list[str]]:
    matched_dict = {}
    for term in terms:
        pattern = re.compile(r'(?<!\d)' + re.escape(term) + r'(?!\d)')
        filtered = []
        for ex in examples:

            # 1) 용어 매칭
            if not (pattern.search(ex)) :
                continue

            # 2) 줄바꿈 검사
            if "\n\n" in ex:
                continue

            # 3) 최대 길이 검사
            if len(ex) > max_len :
                continue

            # 4) “긴 무공백 구간” 검사
            runs = re.findall(r'[가-힣A-Za-z0-9]+', ex)
            if any(len(run) > max_run for run in runs):
                continue

            # 5) 연속된 공백 찾기
            if ' ' * max_space_run in ex:
                continue

            # 6) **‘다.’ 로 끝나지 않는 문장 제외**
            if not ex.strip().endswith("다."):
                continue

            filtered.append(ex)


        # 4) 순서 유지하며 중복 제거
        matched_dict[term] = list(dict.fromkeys(filtered))

    return matched_dict

In [117]:
all_examples = []
for row in weapon_ex['example']:
    try:
        all_examples.append(row)
    except:
        continue

# 2) terms 불러오기
terms = weapon_ex['term'].unique().tolist()

# 3) 필터링 수행
filtered_dict = filter_examples(terms, all_examples, max_run=20, max_space_run= 3,max_len = 200)

# 4) 중복값 제거
for term, exs in filtered_dict.items():
    # dict.fromkeys 이용해 순서는 유지하되 중복 값은 하나만 남김
    filtered_dict[term] = list(dict.fromkeys(exs))

In [118]:
have_example_term = [term for term, exs in filtered_dict.items() if exs]
print(have_example_term) # 용례가 있는 용어
num_terms_with_examples = sum(1 for exs in filtered_dict.values() if exs)
num_examples_with_examples = sum(len(exs) for exs in filtered_dict.values() if exs)
print(f"용례가 있는 용어의 개수: {num_terms_with_examples}개") # 33개로 감소(37개 -> 35개->32개)
print(f"총 용례의 개수: {num_examples_with_examples}개") # 190개로 감소(236개->232개->205개->167개)

['5세대 전투기', 'F-2', 'J-20', 'P-3C', 'SM-3', 'T-50', '견인포', '공격기', '공대공 미사일', '기뢰', '대륙간탄도미사일', '대포병 레이더', '독도함', '랴오닝호', '무인전투기', '미사일 방어체계', '비살상무기', '사드', '상륙돌격장갑차', '알레이버크급 구축함', '어뢰', '장갑차', '장사정포', '전략 폭격기', '전익기', '전자전기', '정찰기', '정찰위성', '지뢰', '천궁', '프레데터', '함포']
용례가 있는 용어의 개수: 32개
총 용례의 개수: 167개


In [119]:
# 결과 확인
for term, exs in filtered_dict.items():
    if exs:
        print(f"{term} ({len(exs)}건):")
        for ex in exs:
            print("  -", ex)
        print()

5세대 전투기 (1건):
  - ■ PL-10 (중국)중국에서 2004년부터 5세대 전투기용으로 개발한  5세대 단거리공대공유도탄이다.

F-2 (1건):
  - 장착기종 으로 F-2, F-15J, F-4EJ에서 운용 중이며, 정밀도 높은 RLG(Ring Laser Gyro) IMU를 탑재하여 LOAL 모드 운용이 가능하다.

J-20 (1건):
  - 중국의 최신전투기인 J-10, J-11, J-20, FC-31 전투기에 장착 예정이며  수출용 버전인 PL-10E도 개발하였다.

P-3C (1건):
  - 하지만, 이스라엘의 ELTA, IAI, Rafael 사의 장비에 의존하여 RF-16펠콘 유인정찰기, 송골매 무인정찰기, P-3C 성능개량기 등을 운용하였다.

SM-3 (3건):
  - SM-3, GBI(Ground Based Intercepter)는 중간단계, 천궁-II, 사드(THAAD), 패트리어트(Patriot)는 종말단계 미사일 방어에 해당한다.
  - 탄소섬유 기반 초고온 내열/내산화 복합소재 기술  	 1	개요초고온 내열 복합소재는 위에 대표 그림에서 나타나 있는 것과 같이 사드(THADD)와 유사한 유도탄 방어무기체계(SM-3)에 필수 핵심부품 소재이다.
  - 2월에 미 해군도 함정발사용 미사일(RIM-161 SM-3)을 발사하여 고장 상태로 궤도를 돌고 있던 자국의 한 정찰위성(USA-193)을 요격하는 데 성공하였다.

T-50 (2건):
  - 최근의 또 다른 사례로는 한국항공우주산업주식회사(KAI)의 T-50 항공기 개발이라 할 수 있다.
  - T-50 항공기 개발비는 약 2조원이었으며, 그 중 10%인 200억을 종합군수지원(IL그렇기S) 개발 비용으로 사용하였기에 후속군수지원을 지속적으로 가능하게 하면서 아랍에미리트와의 수출협상 시 한국 정부차원의 안정적 후속 군수지원 보장에 대한 요구를 충족하게 되며 큰 호평을 받고 있다.

견인포 (2건):
  - 1992년 개념에 대한 최초 입증 후, 2005년에는 화포용 레이저 점화장치(

# 최종 함수

In [120]:
import re
import pandas as pd

# --- 1) CSV 로드 ---
weapon_ex = pd.read_csv(
    '/content/drive/MyDrive/Hanhwa/crawling_project/crawling_project/data/test.csv',
    encoding='utf-8-sig'
)

# --- 2) all_examples 구성 (pdf + example 튜플) ---
all_examples = []
for _, row in weapon_ex.iterrows():
    pdf = row['pdf']
    exs = row['example']
    if isinstance(exs, str):
        try:
            exs = eval(exs)
        except:
            exs = [exs]
    for ex in exs:
        all_examples.append((pdf, ex))

# --- 3) 필터링 함수 (튜플 버전) ---
def filter_examples(terms, examples, max_run=20, max_space_run=5, max_len=100):
    matched = {}
    for term in terms:
        pat = re.compile(r'(?<!\d)' + re.escape(term) + r'(?!\d)')
        hits = []
        for pdf, ex in examples:
            # 1) 용어 매칭
            if not pat.search(ex):
                continue
            # 2) 이중 줄바꿈 제거
            if "\n\n" in ex:
                continue
            # 3) 길이 제한
            if len(ex) > max_len:
                continue
            # 4) 긴 무공백(run) 검사
            runs = re.findall(r'[가-힣A-Za-z0-9]+', ex)
            if any(len(r) > max_run for r in runs):
                continue
            # 5) 연속 공백(run) 검사
            if ' ' * max_space_run in ex:
                continue
            # 6) “다.” 로 끝나지 않는 문장 제거
            if not ex.strip().endswith("다."):
                continue
            # 모두 통과하면, (pdf, example) 기록
            hits.append((pdf, ex))
        # 순서 유지하며 중복 제거
        matched[term] = list(dict.fromkeys(hits))
    return matched

# --- 4) terms 리스트 ---
terms = weapon_ex['term'].unique().tolist()

# --- 5) 필터링 수행 ---
filtered_dict = filter_examples(terms, all_examples,
                                max_run=20,
                                max_space_run=3,
                                max_len=200)

# --- 6) CSV 저장용 DataFrame으로 변환 ---
rows = []
for term, recs in filtered_dict.items():
    for pdf, ex in recs:
        rows.append({
            'term': term,
            'pdf': pdf,
            'example': ex
        })

df_out = pd.DataFrame(rows)

# term 기준 정렬
df_out = df_out.sort_values('term')

# --- 7) CSV로 저장 ---
output_path = "/content/drive/MyDrive/Hanhwa/crawling_project/crawling_project/data/filtered_with_pdf.csv"
df_out.to_csv(output_path, index=False, encoding="utf-8-sig")
print(f"✔ 저장 완료: {output_path}")

✔ 저장 완료: /content/drive/MyDrive/Hanhwa/crawling_project/crawling_project/data/filtered_with_pdf.csv


# csv 저장

In [123]:
def xlsx_to_csv(input_xlsx: str, output_csv: str, sheet_name=0):

    # 1) 엑셀 파일 로드
    df = pd.read_excel(input_xlsx, sheet_name=sheet_name, engine='openpyxl')

    # 2) CSV로 저장 (UTF-8 BOM 포함)
    df.to_csv(output_csv, index=False, encoding='utf-8-sig')
    print(f" '{input_xlsx}' → '{output_csv}' 변환 완료 (encoding='utf-8-sig')")

In [124]:
if __name__ == "__main__":
    input_xlsx = '/content/drive/MyDrive/Hanhwa/crawling_project/crawling_project/data/filtered_with_pdf.xlsx'
    output_csv = '/content/drive/MyDrive/Hanhwa/crawling_project/crawling_project/data/국방과학연구소_무기의세계_용례매핑_전처리후.csv'     # 저장할 csv 파일 경로
    xlsx_to_csv(input_xlsx, output_csv)

 '/content/drive/MyDrive/Hanhwa/crawling_project/crawling_project/data/filtered_with_pdf.xlsx' → '/content/drive/MyDrive/Hanhwa/crawling_project/crawling_project/data/국방과학연구소_무기의세계_용례매핑_전처리후.csv' 변환 완료 (encoding='utf-8-sig')
